In [1]:
import pywren, json, requests, time, bs4
import pandas as pd
from re import findall
import numpy as np

In [2]:
data = pd.read_csv('expert_twitters_complete.csv')
mobile = "https://mobile.twitter.com"
tar_accounts = list(set(data['twitter'])-{'-1', np.NAN})
print(len(tar_accounts))

691


In [3]:
def find_text_urls(tweets):
    temp = tweets.find('div', {'class': 'dir-ltr'})
    try: text = temp.text.strip()
    except AttributeError: return (None, None)
    urls = temp.find('a', {'class':'twitter_external_link'})
    if urls: urls = urls['data-url']
    return text, urls

def extract_texts(tweet_sp):
    t_urls = [find_text_urls(i) for i in tweet_sp.findAll('div', {'class': 'tweet-text'})]
    original_writers = tweet_sp.findAll('td', {'class': 'user-info'})[1:]
    original_writers = [i.find('a')['href'][:-4].strip('/') for i in original_writers]
    time = [i.text.strip() for i in tweet_sp.findAll('td', {'class': 'timestamp'})]
    return list(zip(original_writers, time, t_urls))

def scrape_user_tweets(username):
    base = f"{mobile}/{username}?"
    tar_url = f"{mobile}/{username}"
    results = []
    for i in range(300):
        response = requests.get(tar_url).text
        for i in range(5):
            if response: break
            else: 
                time.sleep(10)
                response = requests.get(tar_url).text
        if not response: break
        soup = bs4.BeautifulSoup(response, "html.parser")
        results.extend(extract_texts(soup))
        cursor = soup.find_all("div", "w-button-more")
        try: init = findall(r'href=(.*?)">', str(cursor))[0].split('?')[1]
        except IndexError: break
        tar_url = base + init
    return results

def scrape_user(username):
    url = f"{mobile}/{username}"
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, "html.parser")
    expert = dict()
    expert['bio'] = soup.find('div', {'class': 'dir-ltr'}).text.strip()
    expert["location"] = soup.find('div', {'class': 'location'}).text.strip()
    stat = soup.findAll('div', {'class': 'statnum'})
    expert['num_tweets'] = stat[0].text.replace(',', '')
    expert['num_following'] = stat[1].text.replace(',', '')
    expert['num_followers'] = stat[2].text.replace(',', '')
    expert['tweets'] = scrape_user_tweets(username)
    
    return {username: expert}

In [4]:
pwex = pywren.default_executor(job_max_runtime = 500)
futures = pwex.map(scrape_user, tar_accounts)
print(len(futures))

691


In [8]:
dones, not_dones = pywren.wait(futures, pywren.ANY_COMPLETED)
print(len(dones), len(not_dones))

691 0


In [9]:
expert_data = dict()
for num, item in enumerate(dones):
    print(num, end = '\r')
    try: expert_data.update(item.result())
    except (IndexError, AttributeError): continue

In [10]:
set(tar_accounts)-expert_data.keys()
## invalid accounts

{'MarshallJenA',
 'sarahfiggatt',
 'shilpaphadke',
 'steven_groves',
 'timmonsroberts',
 'tompepinsky'}

In [12]:
json.dump(expert_data, open('expert_profile_tweets.json', 'w'))

In [30]:
data = pd.read_csv('expert_twitters_complete.csv')
data = data[data['twitter']!='-1']
data = data[~data['twitter'].isin(set(tar_accounts)-expert_data.keys())]

In [31]:
data['twitter_bio'] = data['twitter'].apply(lambda x: expert_data[x]['bio'])
data['location'] = data['twitter'].apply(lambda x: expert_data[x]['location'])
data['num_tweets'] = data['twitter'].apply(lambda x: expert_data[x]['num_tweets'])
data['num_following'] = data['twitter'].apply(lambda x: expert_data[x]['num_following'])
data['num_followers'] = data['twitter'].apply(lambda x: expert_data[x]['num_followers'])

In [33]:
data.to_csv('expert_info_v1.csv', index=False)